### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [1]:
from unsloth import FastModel
import torch
max_seq_length = 512

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length,
    load_in_4bit = True,  # 4 bit quantization to reduce memory
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Gemma3 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 3070 Laptop GPU. Num GPUs = 1. Max memory: 7.658 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update a small amount of parameters!

In [2]:
model = FastModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

## Emotion Representation Pretraining

In [4]:
import pandas as pd
from datasets import Dataset
phase1_dataset = pd.read_json('data/final_datasets/phase1.jsonl', lines=True)
phase1_dataset = Dataset.from_pandas(phase1_dataset)

phase1_dataset = phase1_dataset.train_test_split(
    test_size=0.1,
    seed=3407,
    shuffle=True,
)

We now use `convert_to_chatml` to try converting datasets to the correct format for finetuning purposes!

In [5]:
import json


def convert_phase1_to_chatml(example):
    return {
        "conversations": [
            {
                "role": "system",
                "content": "You are an emotion analysis model. Given a Persian text, output the intensity (0-5) of the following emotions: Anger, Fear, Happiness, Hatred, Sadness, Wonder. Return JSON."
            },
            {
                "role": "user",
                "content": example["text"]
            },
            {
                "role": "assistant",
                "content": json.dumps({
                    "Anger": example['Anger'],
                    "Fear": example['Fear'],
                    "Happiness": example['Happiness'],
                    "Hatred": example['Hatred'],
                    "Sadness": example['Sadness'],
                    "Wonder": example['Wonder']
                })
            }
        ]
    }


phase1_train_dataset = phase1_dataset['train'].map(
    convert_phase1_to_chatml
)

phase1_val_dataset = phase1_dataset['test'].map(
    convert_phase1_to_chatml
)

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [6]:
phase1_train_dataset[100]

{'text': 'آقا پیج این یارو که می گفت معتادا نمیگیرن رو پیدا کردم حالا فردا معروف میشه میزنه تو کار تبریک تولد میگه مهسا جون معتادا تولدتو تبریک میگن ههه ههه',
 'Anger': 2,
 'Fear': 1,
 'Happiness': 0,
 'Hatred': 0,
 'Sadness': 0,
 'Wonder': 0,
 'conversations': [{'content': 'You are an emotion analysis model. Given a Persian text, output the intensity (0-5) of the following emotions: Anger, Fear, Happiness, Hatred, Sadness, Wonder. Return JSON.',
   'role': 'system'},
  {'content': 'آقا پیج این یارو که می گفت معتادا نمیگیرن رو پیدا کردم حالا فردا معروف میشه میزنه تو کار تبریک تولد میگه مهسا جون معتادا تولدتو تبریک میگن ههه ههه',
   'role': 'user'},
  {'content': '{"Anger": 2, "Fear": 1, "Happiness": 0, "Hatred": 0, "Sadness": 0, "Wonder": 0}',
   'role': 'assistant'}]}

We now have to apply the chat template for `Gemma3` onto the conversations, and save it to `text`.

In [7]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

phase1_train_dataset = phase1_train_dataset.map(formatting_prompts_func, batched = True)
phase1_val_dataset = phase1_val_dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Let's see how the chat template did!

In [8]:
phase1_train_dataset[100]['text']

'<start_of_turn>user\nYou are an emotion analysis model. Given a Persian text, output the intensity (0-5) of the following emotions: Anger, Fear, Happiness, Hatred, Sadness, Wonder. Return JSON.\n\nآقا پیج این یارو که می گفت معتادا نمیگیرن رو پیدا کردم حالا فردا معروف میشه میزنه تو کار تبریک تولد میگه مهسا جون معتادا تولدتو تبریک میگن ههه ههه<end_of_turn>\n<start_of_turn>model\n{"Anger": 2, "Fear": 1, "Happiness": 0, "Hatred": 0, "Sadness": 0, "Wonder": 0}<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's train our model. We do 100 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = phase1_train_dataset,
    eval_dataset = phase1_val_dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2, 
        warmup_ratio = 0.05,
        num_train_epochs = 2,
        learning_rate = 5e-5,
        logging_steps = 200,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs/outputs_phase1",
        eval_strategy = "epoch",
        save_strategy = "epoch",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=5):   0%|          | 0/27000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=5):   0%|          | 0/3000 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=6):   0%|          | 0/27000 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/27000 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/3000 [00:00<?, ? examples/s]

Filter (num_proc=6):   0%|          | 0/3000 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [15]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nYou are an emotion analysis model. Given a Persian text, output the intensity (0-5) of the following emotions: Anger, Fear, Happiness, Hatred, Sadness, Wonder. Return JSON.\n\nآقا پیج این یارو که می گفت معتادا نمیگیرن رو پیدا کردم حالا فردا معروف میشه میزنه تو کار تبریک تولد میگه مهسا جون معتادا تولدتو تبریک میگن ههه ههه<end_of_turn>\n<start_of_turn>model\n{"Anger": 2, "Fear": 1, "Happiness": 0, "Hatred": 0, "Sadness": 0, "Wonder": 0}<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                         {"Anger": 2, "Fear": 1, "Happiness": 0, "Hatred": 0, "Sadness": 0, "Wonder": 0}<end_of_turn>\n'

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Laptop GPU. Max memory = 7.658 GB.
1.004 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27,000 | Num Epochs = 2 | Total steps = 3,376
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 6,522,880 of 1,006,408,832 (0.65% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,0.165400,0.164016
2,0.165400,0.163934


Unsloth: Not an error, but Gemma3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [19]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6193.4195 seconds used for training.
103.22 minutes used for training.
Peak reserved memory = 2.971 GB.
Peak reserved memory for training = 1.967 GB.
Peak reserved memory % of max memory = 38.796 %.
Peak reserved memory for training % of max memory = 25.686 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [22]:
messages = [
    {'role': 'system','content':phase1_val_dataset['conversations'][10][0]['content']},
    {"role" : 'user', 'content' : phase1_val_dataset['conversations'][10][1]['content']}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
).removeprefix('<bos>')

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 125,
    temperature = 1, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

{"Anger": 2, "Fear": 2, "Happiness": 0, "Hatred": 1, "Sadness": 0, "Wonder": 2}<end_of_turn>


In [23]:
phase1_val_dataset['conversations'][10]

[{'content': 'You are an emotion analysis model. Given a Persian text, output the intensity (0-5) of the following emotions: Anger, Fear, Happiness, Hatred, Sadness, Wonder. Return JSON.',
  'role': 'system'},
 {'content': 'به نظرم تحلیل مهم داخلی دوپاره شدن جامعه در واکنش به مرگ قاسم سلیمانی نیست بلکه این نکته است که اگر سرداری با درجه و اهمیت او در سال ۶۳ ۶۴ کشته می شد درصد افراد شادمان به ناراحت چه تفاوتی میکرد به عبارت دیگر اعداد خالص مهم نیستند میزان ریزش مهم است',
  'role': 'user'},
 {'content': '{"Anger": 1, "Fear": 0, "Happiness": 0, "Hatred": 0, "Sadness": 1, "Wonder": 0}',
  'role': 'assistant'}]

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Hugging Face's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("outputs/gemma_3_lora_phase1")
tokenizer.save_pretrained("outputs/gemma_3_lora_phase1")

('Phase1_lora/tokenizer_config.json',
 'Phase1_lora/special_tokens_map.json',
 'Phase1_lora/chat_template.jinja',
 'Phase1_lora/tokenizer.model',
 'Phase1_lora/added_tokens.json',
 'Phase1_lora/tokenizer.json')